# Generate QnA synthetic dataset from a Complex PDF using Azure AI Document Intelligence

### Overview

We process the PDF by dividing it into three parts.

-   **Text-heavy** - Text-heavy PDF can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.
-   **Image-heavy** - Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.
-   **Mixed** - After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)

![summary](../imgs/summary-creating-qna-pdf.png)


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = "../../0_lab_preparation"
sys.path.append(os.path.abspath(module_path))

from common import check_kernel
check_kernel()

Kernel: python310-sdkv2


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")
    
print(f"aoai_api_endpoint: {aoai_api_endpoint}")
print(f"aoai_api_key: {aoai_api_key}")
print(f"aoai_api_version: {aoai_api_version}")
print(f"aoai_deployment_name: {aoai_deployment_name}")
print(f"doc_intelligence_endpoint: {doc_intelligence_endpoint}")
print(f"doc_intelligence_key: {doc_intelligence_key}")

aoai_api_endpoint: https://ai-slminnvhubeastus441510290205.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview
aoai_api_key: cc3183fdd2144f01850a60857789c047
aoai_api_version: 2024-08-01-preview
aoai_deployment_name: gpt-4o-mini
doc_intelligence_endpoint: https://slm-innv-eastus-doc.cognitiveservices.azure.com/
doc_intelligence_key: 5IsSs0tvkxzQY8Ff9lB1xYorT1IFWlNNQfkgt4KuixAJaqXNPHKEJQQJ99AJACYeBjFXJ3w3AAALACOGPMrg


## 1. Read & Preprocess PDF file

---


### Split the PDFs into individual pages


In [3]:
import shutil, random
import openai
from langchain_community.document_loaders.csv_loader import CSVLoader
from util.common_utils import get_language_code

raw_data_dir = "../raw_data"
splitted_raw_data_dir = "splitted_raw_data"
file_path = f"{raw_data_dir}/pdf/en-imagenet-training-wrote-by-daekeun.pdf"

DOMAIN = "Distributed training on Cloud"
LANGUAGE = "English" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

Domain: Distributed training on Cloud, Language: English, Language Code: en


(Optional) Only use a poration of the PDF documents for testing. If there are a lot of pages or partial processing is required, cut and save only some pages.


In [4]:
import fitz

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(5, 25)]

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/part{idx}.pdf")

Distinguish between pages composed mainly of text, pages composed primarily of images, and pages composed of mixed text/images.


In [8]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import analyze_pdf_page_content, split_pdf

#file_path = f"{raw_data_dir}/part0.pdf"
analyzed_pdf_result = analyze_pdf_page_content(file_path)
delete_folder_and_make_folder(splitted_raw_data_dir)    

print("### PDF Content Analysis Result:")
for content_type, pages in analyzed_pdf_result.items():
    print(f"{content_type} pages: {pages}")
    split_pdf(file_path, f"{splitted_raw_data_dir}/{content_type}.pdf", pages)

The folder 'splitted_raw_data' and its contents have been deleted.
### PDF Content Analysis Result:
Text pages: [0, 1, 3, 5, 8]
Mixed pages: [2, 4, 6, 7]


In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat
from openai import AzureOpenAI

try:
    client = AzureOpenAI(
        api_key=aoai_api_key,  
        api_version=aoai_api_version,
        base_url=f"{aoai_api_endpoint}/openai/deployments/{aoai_deployment_name}",
        max_retries=1
    )
except (ValueError, TypeError) as e:
    print(e)

try:
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )
except (ValueError, TypeError) as e:
    print(e)    

### Case 1: Mixed page (Images and text mixed appropriately)

After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)


#### Analyze Document

Analyze the document with Azure AI Document Intelligence to extract the text and image information.

-   `crop_image_from_file()`: Crop the image from the PDF file based on the bounding box.
-   `is_bounding_box_larger_than()`: Check if the bounding box is larger than the threshold.
-   `image_complexity()`: Check if the image is complex or simple based on image statistics.
-   `understand_image_with_gpt():` Summarize the image with OpenAI GPT.

![post-process](../imgs/post-process1.png)
![post-process](../imgs/post-process2.png)


In [9]:
if "Mixed" in analyzed_pdf_result:
    pdf_mixed_path = f"{splitted_raw_data_dir}/Mixed.pdf"

    with open(pdf_mixed_path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
            output_content_format=ContentFormat.MARKDOWN 
        )

    result = poller.result()
    md_content = result.content

    #### Updates the content of the figure description (empty content or caption) with the image summary text generated by gpt-4o.
    from util.preprocess import (
        image_complexity, is_bounding_box_larger_than, crop_image_from_file, 
        understand_image_with_gpt, update_figure_description
    )
    output_folder = "pdf_mixed_tmp"
    delete_folder_and_make_folder(output_folder)
    language = LANGUAGE
    max_tokens = 1024
    input_file_path = file_path

    if result.figures:
        print("Figures:")
        for idx, figure in enumerate(result.figures):
            figure_content = ""
            img_description = ""
            print(f"Figure #{idx} has the following spans: {figure.spans}")
            
            for i, span in enumerate(figure.spans):
                print(f"Span #{i}: {span}")
                figure_content += md_content[span.offset:span.offset + span.length]
            print(f"Original figure content in markdown: {figure_content}")

            # Note: figure bounding regions currently contain both the bounding region of figure caption and figure body
            if figure.caption:
                caption_region = figure.caption.bounding_regions
                print(f"\tCaption: {figure.caption.content}")
                print(f"\tCaption bounding region: {caption_region}")
                for region in figure.bounding_regions:
                    if region not in caption_region:
                        print(f"\tFigure body bounding regions: {region}")
                        # To learn more about bounding regions, see https://aka.ms/bounding-region
                        boundingbox = (
                                region.polygon[0],  # x0 (left)
                                region.polygon[1],  # y0 (top)
                                region.polygon[4],  # x1 (right)
                                region.polygon[5]   # y1 (bottom)
                            )

                        if is_bounding_box_larger_than(boundingbox):
                            print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")
                            cropped_image = crop_image_from_file(pdf_mixed_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                            if image_complexity(cropped_image)[0] == "Complex":
                                # Get the base name of the file
                                base_name = os.path.basename(input_file_path)
                                # Remove the file extension
                                file_name_without_extension = os.path.splitext(base_name)[0]

                                output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                                cropped_image_filename = os.path.join(output_folder, output_file)

                                cropped_image.save(cropped_image_filename)
                                print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")

                                try: 
                                    image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, "", max_tokens=max_tokens, language=language)
                                except openai.BadRequestError as e:
                                    print(f"BadRequestError: {e}")
                                    image_summarization = ""
                                img_description += image_summarization

                                print(f"\tDescription of figure {idx}: {img_description}")
                            else:
                                print(f'simple image at idx {idx}')

            else:
                print("\tNo caption found for this figure.")
                for region in figure.bounding_regions:
                    print(f"\tFigure body bounding regions: {region}")
                    # To learn more about bounding regions, see https://aka.ms/bounding-region
                    boundingbox = (
                            region.polygon[0],  # x0 (left)
                            region.polygon[1],  # y0 (top
                            region.polygon[4],  # x1 (right)
                            region.polygon[5]   # y1 (bottom)
                        )

                    if is_bounding_box_larger_than(boundingbox):                    
                        print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")

                        cropped_image = crop_image_from_file(input_file_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                        if image_complexity(cropped_image)[0] == "Complex":
                            # Get the base name of the file
                            base_name = os.path.basename(input_file_path)
                            # Remove the file extension
                            file_name_without_extension = os.path.splitext(base_name)[0]

                            output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                            cropped_image_filename = os.path.join(output_folder, output_file)
                            # cropped_image_filename = f"data/cropped/image_{idx}.png"
                            cropped_image.save(cropped_image_filename)
                            print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")

                            try:
                                image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, "", max_tokens=max_tokens, language=language)
                            except openai.BadRequestError as e:
                                print(f"BadRequestError: {e}")
                                image_summarization = ""
                            img_description += image_summarization
                            print(f"\tDescription of figure {idx}: {img_description}")
                        else:
                            print(f'simple image at idx {idx}')

            
            md_content = update_figure_description(md_content, img_description, idx)    

The folder 'pdf_mixed_tmp' and its contents have been deleted.
Figures:
Figure #0 has the following spans: [{'offset': 7309, 'length': 38}]
Span #0: {'offset': 7309, 'length': 38}
Original figure content in markdown: 
<figure>

![](figures/0)

</figure>


	No caption found for this figure.
	Figure body bounding regions: {'pageNumber': 4, 'polygon': [1.2863, 1.3971, 7.2608, 1.4262, 7.2445, 4.4455, 1.2693, 4.4162]}
is_bounding_box_larger_than - width: 5.958200000000001, height: 3.0484
	Figure body bounding box in (x0, y0, x1, y1): (1.2863, 1.3971, 7.2445, 4.4455)
	Figure 0 cropped and saved as pdf_mixed_tmp/en-imagenet-training-wrote-by-daekeun_cropped_image_0.png
	Description of figure 0: The image shows a code snippet for setting up and downloading the ImageNet dataset by exporting a username and access key. Below are the steps mentioned in the code block:

1. Export the ImageNet username:
   ```
   export IMAGENET_USERNAME=[YOUR_USERNAME]
   ```

2. Export the ImageNet access key:
   

Generate chunks for mixed pages


In [10]:
if "Mixed" in analyzed_pdf_result:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import re

    text_splitter = RecursiveCharacterTextSplitter(
        separators=[
            r'<!-- PageNumber="\d+" -->',
            r"\n\n",
            r"\n",
            " ",
            ".",
            "",
        ],   
        is_separator_regex = True,    
        chunk_size=2000,
        chunk_overlap=200,
    )

    mixed_chunks = text_splitter.split_text(md_content)
    print("Length of splits (mixed case): " + str(len(mixed_chunks)))
else:
    mixed_chunks = []

Length of splits (mixed case): 8


In [11]:
mixed_chunks

['<!-- PageHeader="24\\. 7. 22. 오전 9:52" -->\n\n<!-- PageHeader="[Hands-on] Fast Training ImageNet on on-demand EC2 GPU instances with Horovod" -->\n\n. Format the EBS volume, mount it on /data , and then change the owner to ec2-user: ec2-user . You may refer to\n\nhttps://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using- volumes.html if you do not know how to mount it.\n\n. Download MXNet repository and TensorFlow models repository.\n\n$ cd /data $ git clone https://github.com/tensorflow/models.git $ git clone https://github.com/apache/incubator-mxnet.git # or, you can just type \'pip install mxnet\\`\n\n· [Optional] For your convenience, use symbolic link such that:\n\n|| [ec2-user@ip-172-31-34-246 data]$ ls -1 || | | |||\n| 합계 13864 ||| | | |||\n| - | - | - | - | - | - | - | - |\n| drwxrwxr-x 4 ec2-user ec2-user ||| 37 | 9월 | 16 02:20 im2rec |||\n||| rwxrwxrwx 1 ec2-user ec2-user   |||| 41 9% 10 02:38 imagenet -> /data/models/research/inception/inception    ||\n||| drwxrwxr-x 7

### Case 2: Text-heavy

Text-heavy PDFs can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.


In [12]:
if "Text" in analyzed_pdf_result:
    from langchain_community.document_loaders.pdf import PyMuPDFLoader
    from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

    pdf_text_path = f"{splitted_raw_data_dir}/Text.pdf"
    loader = PyMuPDFLoader(pdf_text_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, 
        chunk_overlap=200
    )

    text_chunks = text_splitter.split_documents(documents)

    for idx, chunk in enumerate(text_chunks):
        print(f"Chunk {idx}\n{chunk}")
        print("="*80)
        if idx == 2:
            break

    text_chunks = [d.page_content for d in text_chunks]
    print("Length of splits (text-heay case): " + str(len(text_chunks)))
else:
    text_chunks = []

Chunk 0
page_content='[Hands-on] Fast Training
ImageNet on on-demand EC2
GPU instances with Horovod
💻
Author: Daekeun Kim (daekeun@amazon.com)
Goal
This document is for people who need distributed GPU training using Horovod for
experimental purposes. Many steps are similar to what mentioned in Julien
Simon’s article(
) and AWS
Documentation(
). So I recommend you to view these articles first. If there
are some things that aren’t going well (e.g., Downloading the dataset does not
work, How to convert the raw data to the TFRecord feature set?, How to fix the
error ModuleNotFoundError: No module named 'cv2'? ) please refer this
document.
https://medium.com/@julsimon/imagenet-part-1-going-on-an-
adventure-c0a62976dc72
https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-
horovod-tensorflow.html
Introduction
For data preparation and data transformation, we do not need to use a GPU
instance such as p2 and p3. Instead, we can start much cheaper instances like 
t2.large  instance with 1.

In [13]:
len(text_chunks)

9

### Case 3: Image-heavy

Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.

### Preprocess Image


In [28]:
if "Image" in analyzed_pdf_result:
    import fitz
    from glob import glob

    image_dir = "./pdf_image_tmp"
    delete_folder_and_make_folder(image_dir) 

    pdf_image_path = f"{splitted_raw_data_dir}/Image.pdf"
    doc = fitz.open(pdf_image_path)
    #clip_x, clip_y = 10, 45
    clip_x, clip_y = 10, 10

    for i, page in enumerate(doc):
        x, y, w, h = page.rect
        clip = fitz.Rect(x+clip_x, y+clip_y, w-clip_x, h-clip_y)
        page.set_cropbox(clip)
        pix = page.get_pixmap()
        pix.save(f"{image_dir}/page_{i:03d}.jpg")

    images = sorted(glob(os.path.join(image_dir, "*.jpg")))

In [17]:
if "Images" in analyzed_pdf_result:
    print(f"extracted {len(images)} images")
else:
    print(f"no image heavy documents")

no image heavy documents


In [18]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI

max_tokens = 1024
llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=max_tokens,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name
)

human_prompt_main = f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."

system_prompt = "You are an assistant tasked with describing table or image, specialized in Smartphone product."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": human_prompt_main
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()

In [19]:
%%time
if "Image" in analyzed_pdf_result:
    from util.preprocess import encode_image_base64
    #images = glob(os.path.join(image_path, "*.jpg"))
    base64_images = [encode_image_base64(img_path) for img_path in images]
    image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 8})
    image_summaries = remove_short_sentences(image_summaries)
    print("Length of image_summaries (image-heavy case): " + str(len(image_summaries)))
else:
    print("No image heavy doc in analyzed pdfs")
    image_summaries = []


No image heavy doc in analyzed pdfs
CPU times: user 102 μs, sys: 17 μs, total: 119 μs
Wall time: 124 μs


Let's see all chunks.

In [20]:
print(f"""
      All chunks from analyzed PDF documents:
      Mixed chunks: {len(mixed_chunks)} chunks. 
      Text heavy chunks: {len(text_chunks)} chunks.
      Images extracted: {len(image_summaries)}
      """)


      All chunks from analyzed PDF documents:
      Mixed chunks: 8 chunks. 
      Text heavy chunks: 9 chunks.
      Images extracted: 0
      


## 2. Construct QnA Pairs

---

### Option 1.

Leverage the `azure-ai-generative` package. The `QADataGenerator` class in this package makes it easy to generate QnA synthetic questions. However, using this class as is has the disadvantage of not being able to use custom prompts, so we inherited from it and created the `CustomQADataGenerator` class.


In [31]:
from util.qa import CustomQADataGenerator
model_config = {
    "deployment": aoai_deployment_name,
    "model": "gpt-4o",
    "max_tokens": 2000,
}

qa_generator = CustomQADataGenerator(model_config=model_config, templates_dir=f"./prompt_template/{LANGUAGE_CODE}")

In [32]:
import asyncio
from collections import Counter
from typing import Dict
import os
from azure.ai.generative.synthetic.qa import QAType
concurrency = 6  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

#qa_type = QAType.CONVERSATION
qa_type = QAType.LONG_ANSWER

async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(
            text=text,
            qa_type=qa_type,
            num_questions=3,  # Number of questions to generate per text
        )

In [ ]:
input_batch = mixed_chunks + text_chunks + image_summaries
results = await asyncio.gather(*[generate_async(text) for text in input_batch], return_exceptions=True)

question_answer_list = []
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])

print("Successfully generated QAs")

### Option 2.

You write the entire sequence of code to create a QnA dataset without using a separate toolkit.


In [21]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template(LANGUAGE)

chain = prompt | llm | parser

In [22]:
input_batch = []

for doc in mixed_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in text_chunks:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)

for doc in image_summaries:
    dic = {"context": doc, "domain": DOMAIN, "num_questions": "3"}
    input_batch.append(dic)        

In [23]:
input_batch

[{'context': '<!-- PageHeader="24\\. 7. 22. 오전 9:52" -->\n\n<!-- PageHeader="[Hands-on] Fast Training ImageNet on on-demand EC2 GPU instances with Horovod" -->\n\n. Format the EBS volume, mount it on /data , and then change the owner to ec2-user: ec2-user . You may refer to\n\nhttps://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using- volumes.html if you do not know how to mount it.\n\n. Download MXNet repository and TensorFlow models repository.\n\n$ cd /data $ git clone https://github.com/tensorflow/models.git $ git clone https://github.com/apache/incubator-mxnet.git # or, you can just type \'pip install mxnet\\`\n\n· [Optional] For your convenience, use symbolic link such that:\n\n|| [ec2-user@ip-172-31-34-246 data]$ ls -1 || | | |||\n| 합계 13864 ||| | | |||\n| - | - | - | - | - | - | - | - |\n| drwxrwxr-x 4 ec2-user ec2-user ||| 37 | 9월 | 16 02:20 im2rec |||\n||| rwxrwxrwx 1 ec2-user ec2-user   |||| 41 9% 10 02:38 imagenet -> /data/models/research/inception/inception    ||\n||| 

In [24]:
%%time
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

CPU times: user 520 ms, sys: 28.8 ms, total: 549 ms
Wall time: 2min 5s


In [25]:
qa_pair

[{'QUESTION': 'How do you format and mount an EBS volume on /data and change its owner to ec2-user?',
  'ANSWER': 'To format and mount an EBS volume on /data and change its owner to ec2-user, you can refer to the AWS documentation at https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using-volumes.html.'},
 {'QUESTION': 'What is the first step to download ImageNet according to the provided method?',
  'ANSWER': 'The first step to download ImageNet according to the provided method is to go to http://www.image-net.org/, sign up, and get your own username and access key.'},
 {'QUESTION': "What is the size of the 'ILSVRC2012_img_train.tar' file?",
  'ANSWER': "The size of the 'ILSVRC2012_img_train.tar' file is 147897477120 bytes."},
 [{'QUESTION': "What are the recommended EC2 instance types for distributed GPU training using Uber's Horovod or Tensorflow's DistributedStrategy?",
   'ANSWER': "The recommended EC2 instance types for distributed GPU training using Uber's Horovod or Tenso

In [29]:
# if using only option 2, run this
question_answer_list = qa_pair

## 3. Save to jsonl

---

If you want to augment dataset, you can try Evovle-Instruct or other data augmentation techniques.<br>
Please refer to `../evolve-instruct` and `../glan-instruct` for more details.


In [30]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "imagenet-training-summary"
# if using option 2

oai_qa_pair = convert_to_oai_format(question_answer_list, system_prompt_msg=system_prompt_msg)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")

### Clean up


In [41]:
!rm -rf {splitted_raw_data_dir} pdf_image_tmp pdf_mixed_tmp outputs_tmp images